### Experiment with MongoEngine

The ODM we use to access MongoDB

In [1]:
from dexter.DB import DB, Account, Entry, Transaction, RegExp, Document, Tag

from datetime import date, datetime, timedelta

Open the database:

In [2]:
DB.init()

In [3]:
# DB.open('pytest')
DB.open('flat')

Make an account:

In [4]:
acct = Account(name='equity', category='equity')

Save it:

In [5]:
acct.save()

<Account: <Ac equity equity None>>

If we open that DB with `mongosh` we should see the account.

```
$ mongosh

test> use foo
switched to db foo

foo> db.account.find()
[
  {
    _id: ObjectId('67c61fa19d0161a19b80469e'),
    name: 'equity',
    group: 'equity'
  }
]
```

It worked!  🎉

### Contents of a Collection

In [6]:
Account.objects

[<Account: <Ac equity equity None>>, <Account: <Ac assets:bank:checking assets checking>>, <Account: <Ac assets:bank:savings assets savings>>, <Account: <Ac expenses:car expenses None>>, <Account: <Ac expenses:car:payment expenses None>>, <Account: <Ac expenses:car:fuel expenses None>>, <Account: <Ac expenses:food expenses None>>, <Account: <Ac expenses:food:groceries expenses groceries>>, <Account: <Ac expenses:food:restaurant expenses dining>>, <Account: <Ac expenses:home expenses None>>, <Account: <Ac expenses:home:household expenses household>>, <Account: <Ac expenses:home:mortgage expenses mortgage>>, <Account: <Ac expenses:travel expenses None>>, <Account: <Ac income:yoyodyne income None>>, <Account: <Ac liabilities:chase:visa liabilities visa>>, <Account: <Ac equity equity None>>]

In [7]:
Account.objects[0]

<Account: <Ac equity equity None>>

In [8]:
acct = Account.objects[0]

In [9]:
acct.name

'equity'

In [10]:
acct['name']

'equity'

### Low Level API

We can also connect to the DB directly to use the `pymongo` library, _e.g._ to get collection names.

After calling `DB.open` we can get a reference to the client and the current database using static vars of the module:

In [11]:
DB.server

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None))

In [12]:
DB.server.list_database_names()

['admin', 'config', 'dev', 'errors', 'finances', 'local', 'newdates', 'pytest']

In [13]:
DB.database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest')

In [14]:
db = DB.database

In [15]:
db.account

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [16]:
db.account.find_one()

{'_id': ObjectId('68d58be4d827ac4e7ac33e5d'),
 'name': 'equity',
 'category': 'equity'}

In [17]:
for c in db.list_collections():
    print(c)

{'name': 'account', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'Z6\xa1,/\xfeA\x82\x86(\x88\xccD\x87\xd4\xd8', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'dexter', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\xadp53*\x00Hk\xb1*\xa2\x00\xdc\xf8\xf6\x80', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'entry', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\xb7\xc2\xdd\x8f\xd1\xd6E\xc4\x85J\x1fr\x19D(e', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}
{'name': 'transaction', 'type': 'collection', 'options': {}, 'info': {'readOnly': False, 'uuid': Binary(b'\xfay\xf4-\x03.@\xa7\x97\rk\x876\x1d\xc9\xcb', 4)}, 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}


In [18]:
for name in db.list_collection_names():
    print(name)

account
dexter
entry
transaction


In [19]:
db['account']

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [20]:
db['account'].find_one()

{'_id': ObjectId('68d58be4d827ac4e7ac33e5d'),
 'name': 'equity',
 'category': 'equity'}

In [21]:
for obj in db['account'].find():
    print(obj)

{'_id': ObjectId('68d58be4d827ac4e7ac33e5d'), 'name': 'equity', 'category': 'equity'}
{'_id': ObjectId('68d58be4d827ac4e7ac33e5e'), 'name': 'assets:bank:checking', 'category': 'assets', 'abbrev': 'checking', 'parser': 'occu'}
{'_id': ObjectId('68d58be4d827ac4e7ac33e5f'), 'name': 'assets:bank:savings', 'category': 'assets', 'abbrev': 'savings', 'parser': 'occu'}
{'_id': ObjectId('68d58be4d827ac4e7ac33e60'), 'name': 'expenses:car', 'category': 'expenses'}
{'_id': ObjectId('68d58be4d827ac4e7ac33e61'), 'name': 'expenses:car:payment', 'category': 'expenses'}
{'_id': ObjectId('68d58be4d827ac4e7ac33e62'), 'name': 'expenses:car:fuel', 'category': 'expenses'}
{'_id': ObjectId('68d58be4d827ac4e7ac33e63'), 'name': 'expenses:food', 'category': 'expenses'}
{'_id': ObjectId('68d58be4d827ac4e7ac33e64'), 'name': 'expenses:food:groceries', 'category': 'expenses', 'abbrev': 'groceries'}
{'_id': ObjectId('68d58be4d827ac4e7ac33e65'), 'name': 'expenses:food:restaurant', 'category': 'expenses', 'abbrev': 'd

In [22]:
DB.database.regexp

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'regexp')

In [23]:
db.regexp

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'regexp')

In [24]:
res = db.regexp.delete_many({})

In [25]:
res.deleted_count

0

In [26]:
RegExp

dexter.DB.RegExp

In [27]:
RegExp.objects

[]

In [28]:
RegExp.objects.delete()

0

In [29]:
RegExp.objects

[]

### From Low Level to High Level

Question:  given a collection name ("account") can we find the corresponding MongoEngine class (Account)?

In [30]:
Document

mongoengine.document.Document

In [31]:
Document.__subclasses__()

[mongoengine.document.DynamicDocument,
 dexter.DB.Dexter,
 dexter.DB.Account,
 dexter.DB.Entry,
 dexter.DB.Transaction,
 dexter.DB.RegExp]

In [32]:
[cls for cls in Document.__subclasses__() if hasattr(cls, 'objects')]

[dexter.DB.Dexter,
 dexter.DB.Account,
 dexter.DB.Entry,
 dexter.DB.Transaction,
 dexter.DB.RegExp]

In [33]:
Account._meta

{'abstract': False,
 'max_documents': None,
 'max_size': None,
 'ordering': [],
 'indexes': [],
 'id_field': 'id',
 'index_background': False,
 'index_opts': None,
 'delete_rules': None,
 'allow_inheritance': None,
 'collection': 'account',
 'index_specs': []}

In [34]:
for cls in Document.__subclasses__():
    if not hasattr(cls, 'objects'):
        continue
    print(cls._meta['collection'], cls)

dexter <class 'dexter.DB.Dexter'>
account <class 'dexter.DB.Account'>
entry <class 'dexter.DB.Entry'>
transaction <class 'dexter.DB.Transaction'>
reg_exp <class 'dexter.DB.RegExp'>


### The Big Picture

Use the high level API when working with data.  MongoEngine converts the documents into objects (which is something we'd be doing ourselves if we didn't use it).

Use the low level API for collective operations: exporting, importing, ...

**NOTE**  It's possible to get a document using the low level API, as shown above, but it will be a `dict`, not a model instance.

### Transactions

In [35]:
t = Transaction(description='hi', comment='aloha')

In [36]:
t.description

'hi'

Nice -- the list fields are initially empty.

In [37]:
t.tags

[]

In [38]:
t.entries

[]

### Entries

In [39]:
e = Entry(uid='xxx', column='credit', date='2025-03-05', amount=1000, account='unknown')

In [40]:
type(e)

dexter.DB.Entry

In [41]:
e

<Entry: <En 2025-03-05 unknown -$1000.0 []>>

In [42]:
e.column

<Column.cr: 'credit'>

In [43]:
e.column.opposite()

<Column.dr: 'debit'>

In [44]:
e.column.opposite().opposite()

<Column.cr: 'credit'>

In [45]:
e.amount

1000.0

In [46]:
e.hash

'6fce51cdae9a1803b7c8d26e12244edc'

In [47]:
len(e.hash)

32

In [48]:
{e.uid for e in Entry.objects}

{'027e4381664e8fd7cbd6f05862ec257c',
 '035404716c3d4e12fbfd3ba334d5e53b',
 '0d61d5b0fe349ca19c13af086a6ba548',
 '144cb37ff850ddd7c5a6ec0029a058f1',
 '19bbf4989ebaaa02d2417d82f0c879f5',
 '1a0f830c763239d58106bb853409cf2b',
 '220a8516a912d5210c388d543b45fbf9',
 '27916822f82b782a8d47e8dedbfcdf45',
 '2a11737320d943052fcc4dd47f303c4f',
 '360ffdfbe30c25d655ccb6408b0ecee3',
 '468fb03a0f0ebc2d9bfa8a06d34e5e98',
 '489e4befd91753d5dc49bfcc221b5676',
 '60778935e3980c4bebe69fe6843a52ba',
 '6a3e9221e29c2ee2f860c51a273aea96',
 '6acfe94644a2ef9cd2db91166f5a97e6',
 '6c9b71192dc84ed15c49c661c086c966',
 '7081fda3304e2a8966db1205ccbecc14',
 '727d5543a99a71772bd1f0a636748675',
 '73ba3b9b44bfbb23908fdf99795e97ff',
 '75da88f351d54db14abbb9ee61711c68',
 '9a40bfe53c5b95599c2206a9e3e0aee7',
 'a1169acb9de50c2cc6eea2d96348bca6',
 'a3458e99dfa6d98c17592447d0688327',
 'a6c240d6ecd12f69b0425c67317d046a',
 'b7ed10e1dbb1d2194075389ae155b710',
 'bc2bb65cedb51ac170d666350c8aaa9a',
 'bca93ab95fa5f3d678b3283249481f94',
 

In [49]:
s = set()
for e in Entry.objects:
    if e in s:
        print(e.date, e.amount, e.description)
    s.add(e.uid)

In [50]:
len(s)

40

In [51]:
len(Entry.objects)

40

In [52]:
lst = sorted([e.uid for e in Entry.objects])

In [53]:
len(lst)

40

In [54]:
dup = []
for i in range(len(lst)-1):
    if lst[i] == lst[i+1]:
        dup.append(lst[i])

In [55]:
dup

[]

In [56]:
for e in Entry.objects:
    if e.uid in dup:
        print(e.date, e.amount, e.account, e.description)

### Tags

In [11]:
e = Entry.objects[0]

In [12]:
e.description

''

In [13]:
e.tags

[]

In [14]:
# e.tags.append(Tag.U)
e.tags.append('#unpaired')

In [15]:
e.save()

<Entry: <En 2024-12-31 assets:occu:buckets +$108038.57 ['#unpaired']>>

In [16]:
e.tags

['#unpaired']

In [63]:
e.note = "hello"

In [64]:
e.save()

<Entry: <En 2024-01-02 assets:bank:checking +$5000.0 [<Tag.U: '#unpaired'>]>>

In [17]:
len(Entry.objects)

761

In [ ]:
# list(Entry.objects(tags=Tag.U))

[<Entry: <En 2024-01-02 assets:bank:checking +$5000.0 [<Tag.U: '#unpaired'>]>>]

In [18]:
list(Entry.objects(tags='#unpaired'))

[<Entry: <En 2024-12-31 assets:occu:buckets +$108038.57 ['#unpaired']>>]

In [ ]:
# list(Entry.objects(tags__exact=Tag.U))

[<Entry: <En 2024-01-02 assets:bank:checking +$5000.0 [<Tag.U: '#unpaired'>]>>]

In [24]:
list(Entry.objects(tags__iregex='unp'))

[<Entry: <En 2024-12-31 assets:occu:buckets +$108038.57 ['#unpaired']>>]

In [25]:
len(Entry.objects(tags__contains='unpaired'))

1

In [26]:
len(Entry.objects(tags__not__contains='unpaired'))

760

### References

The big test -- can we add that Entry to the transaction?

In [65]:
t.entries.append(e)

In [66]:
t.entries

[<Entry: <En 2024-01-02 assets:bank:checking +$5000.0 [<Tag.U: '#unpaired'>, <Tag.U: '#unpaired'>, <Tag.U: '#unpaired'>, <Tag.U: '#unpaired'>]>>]

Yes!  🎉

### Misc Commands

In [67]:
db.stats

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'stats')

In [68]:
db.stats.find_one

<bound method Collection.find_one of Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'stats')>

In [69]:
db.list_collection_names()

['transaction', 'entry', 'account', 'dexter', 'bar', 'foo']

In [70]:
db.command('count','account')

{'n': 19, 'ok': 1.0}

In [71]:
db.command('hello')

{'isWritablePrimary': True,
 'topologyVersion': {'processId': ObjectId('68791171a4321630896c2581'),
  'counter': 0},
 'maxBsonObjectSize': 16777216,
 'maxMessageSizeBytes': 48000000,
 'maxWriteBatchSize': 100000,
 'localTime': datetime.datetime(2025, 8, 2, 3, 38, 47, 3000),
 'logicalSessionTimeoutMinutes': 30,
 'connectionId': 3785,
 'minWireVersion': 0,
 'maxWireVersion': 25,
 'readOnly': False,
 'ok': 1.0}

In [72]:
db.command('hostInfo')

{'system': {'currentTime': datetime.datetime(2025, 8, 2, 3, 38, 47, 8000),
  'hostname': 'cthulhu.local',
  'cpuAddrSize': 64,
  'memSizeMB': 65536,
  'memLimitMB': 65536,
  'numCores': 12,
  'numCoresAvailableToProcess': 12,
  'numPhysicalCores': 12,
  'numCpuSockets': 1,
  'cpuArch': 'arm64',
  'numaEnabled': False,
  'numNumaNodes': 1},
 'os': {'type': 'Darwin', 'name': 'Mac OS X', 'version': '24.5.0'},
 'extra': {'versionString': 'Darwin Kernel Version 24.5.0: Tue Apr 22 19:54:25 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T6020',
  'alwaysFullSync': 0,
  'nfsAsync': 0,
  'model': 'Mac14,13',
  'cpuString': 'Apple M2 Max',
  'pageSize': 16384,
  'scheduler': 'edge'},
 'ok': 1.0}

In [73]:
db.command('ping')

{'ok': 1.0}

### Transaction Class

We add extra info in the form of static vars.  Make sure it doesn't mess with field specs.

In [181]:
Transaction.order_by

{'description': 'description',
 'date': 'pdate',
 'amount': 'pamount',
 'debit': 'pdebit',
 'credit': 'pcredit'}

### Fetch Transactions

Specify constraints on transactions

In [74]:
Transaction.objects

[<Transaction: <Tr 2024-01-02 income:yoyodyne -> assets:bank:checking $5000.0 Jan paycheck []>>, <Transaction: <Tr 2024-01-02 expenses:food/expenses:home/expenses:car/expenses:travel/equity -> income:yoyodyne $5000.0 fill envelopes ['#budget']>>, <Transaction: <Tr 2024-02-02 income:yoyodyne -> assets:bank:checking $5000.0 Feb paycheck []>>, <Transaction: <Tr 2024-02-02 expenses:food/expenses:home/expenses:car/expenses:travel/equity -> income:yoyodyne $5000.0 fill envelopes ['#budget']>>, <Transaction: <Tr 2024-01-05 assets:bank:checking -> expenses:car:payment $500.0 car payment []>>, <Transaction: <Tr 2024-02-05 assets:bank:checking -> expenses:car:payment $500.0 car payment []>>, <Transaction: <Tr 2024-01-10 liabilities:chase:visa -> expenses:car:fuel $50.0 Shell Oil []>>, <Transaction: <Tr 2024-02-26 liabilities:chase:visa -> expenses:car:fuel $60.0 Shell Oil []>>, <Transaction: <Tr 2024-01-04 assets:bank:checking -> expenses:home:mortgage $1800.0 Rocket Mortgage []>>, <Transaction:

In [75]:
Transaction.objects(description='Safeway')

[<Transaction: <Tr 2024-01-07 assets:bank:checking -> expenses:food:groceries $75.0 Safeway []>>, <Transaction: <Tr 2024-01-21 liabilities:chase:visa -> expenses:food:groceries $175.0 Safeway []>>, <Transaction: <Tr 2024-02-07 assets:bank:checking -> expenses:food:groceries $75.0 Safeway []>>, <Transaction: <Tr 2024-02-21 liabilities:chase:visa -> expenses:food:groceries $75.0 Safeway []>>]

In [76]:
t = Transaction.objects(description='Safeway')[0]

In [77]:
t.pdate

datetime.date(2024, 1, 7)

In [78]:
t['pdate']

datetime.date(2024, 1, 7)

In [79]:
for t in Transaction.objects(description='Safeway'):
    for e in t.entries:
        print(e.date, e.account, e.amount, e.column)

2024-01-07 expenses:food:groceries 75.0 Column.dr
2024-01-07 assets:bank:checking 75.0 Column.cr
2024-01-21 expenses:food:groceries 175.0 Column.dr
2024-01-21 liabilities:chase:visa 175.0 Column.cr
2024-02-07 expenses:food:groceries 75.0 Column.dr
2024-02-07 assets:bank:checking 75.0 Column.cr
2024-02-21 expenses:food:groceries 75.0 Column.dr
2024-02-21 liabilities:chase:visa 75.0 Column.cr


In [80]:
for t in Transaction.objects(description='Safeway'):
    print(t.accounts)

{'expenses:food:groceries', 'assets:bank:checking'}
{'expenses:food:groceries', 'liabilities:chase:visa'}
{'expenses:food:groceries', 'assets:bank:checking'}
{'expenses:food:groceries', 'liabilities:chase:visa'}


In [81]:
for t in Transaction.objects(description='Safeway'):
    print(t.pamount)

75.0
175.0
75.0
75.0


In [82]:
for t in Transaction.objects(description='Safeway'):
    print(t.pdate, type(t.pdate))

2024-01-07 <class 'datetime.date'>
2024-01-21 <class 'datetime.date'>
2024-02-07 <class 'datetime.date'>
2024-02-21 <class 'datetime.date'>


In [83]:
for t in Transaction.objects(description='Safeway'):
    print(t.originals)

assigned/read from CSV
big party this weekend/
/
/


In [84]:
lst = list(Transaction.objects(description='Safeway'))

In [85]:
lst[1].comment

''

In [86]:
lst[1].pamount

175.0

In [87]:
lst[1].pdate

datetime.date(2024, 1, 21)

In [88]:
lst[1].pdate - timedelta(days=1)

datetime.date(2024, 1, 20)

In [89]:
list(Transaction.objects(pamount__lt=175.0))

[<Transaction: <Tr 2024-01-10 liabilities:chase:visa -> expenses:car:fuel $50.0 Shell Oil []>>,
 <Transaction: <Tr 2024-02-26 liabilities:chase:visa -> expenses:car:fuel $60.0 Shell Oil []>>,
 <Transaction: <Tr 2024-01-07 assets:bank:checking -> expenses:food:groceries $75.0 Safeway []>>,
 <Transaction: <Tr 2024-02-07 assets:bank:checking -> expenses:food:groceries $75.0 Safeway []>>,
 <Transaction: <Tr 2024-02-21 liabilities:chase:visa -> expenses:food:groceries $75.0 Safeway []>>,
 <Transaction: <Tr 2024-03-10 liabilities:chase:visa -> expenses:home:household $50.0 Home Depot []>>,
 <Transaction: <Tr 2024-03-12 expenses:home:household -> liabilities:chase:visa $25.0 Home Depot []>>]

In [90]:
for t in Transaction.objects(pdate=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

2024-01-02 5000.0 assets:bank:checking income:yoyodyne
2024-01-02 5000.0 income:yoyodyne expenses:food/expenses:home/expenses:car/expenses:travel/equity


In [91]:
for t in Transaction.objects(pdate__lte=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

2024-01-02 5000.0 assets:bank:checking income:yoyodyne
2024-01-02 5000.0 income:yoyodyne expenses:food/expenses:home/expenses:car/expenses:travel/equity


### Operators

In [92]:
for t in Transaction.objects(description__gte='Safeway'):
    print(t.pdate, t.description)

2024-01-02 fill envelopes
2024-02-02 fill envelopes
2024-01-05 car payment
2024-02-05 car payment
2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


In [93]:
for t in Transaction.objects(description__regex='^S'):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


The operator automatically applies to list elements.

In [94]:
for t in Transaction.objects(description__regex=r'\s'):
    print(t.pdate, t.description, t.pamount)

2024-01-02 Jan paycheck 5000.0
2024-01-02 fill envelopes 5000.0
2024-02-02 Feb paycheck 5000.0
2024-02-02 fill envelopes 5000.0
2024-01-05 car payment 500.0
2024-02-05 car payment 500.0
2024-01-10 Shell Oil 50.0
2024-02-26 Shell Oil 60.0
2024-01-04 Rocket Mortgage 1800.0
2024-02-04 Rocket Mortgage 1800.0
2024-03-10 Home Depot 50.0
2024-03-12 Home Depot 25.0


For compound constraints we need another class from MongoEngine.

In [95]:
from mongoengine.queryset.visitor import Q

In [96]:
for t in Transaction.objects(Q(description__regex=r'^S')):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


In [97]:
for t in Transaction.objects(Q(description__regex=r'^S') & Q(description__regex=r'\s')):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil


### QuerySet

In [98]:
for a in Account.nominal_accounts:
    print(a.name)

### Combining Query Elements

In [99]:
q = Q(description__regex=r'^S')

In [100]:
q

Q(**{'description__regex': '^S'})

In [101]:
type(q)

mongoengine.queryset.visitor.Q

In [102]:
p = Q(description__regex=r'\s')

In [103]:
p & q

(Q(**{'description__regex': '\\s'}) & Q(**{'description__regex': '^S'}))

In [104]:
for t in Transaction.objects(p & q):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil


Create Q object using dictionaries

In [105]:
dct = {'description__regex': r'^S'}

In [106]:
Q(**dct)

Q(**{'description__regex': '^S'})

Can an object have multiple constraints?

In [107]:
dct = {'description__regex': r'^S', 'pamount__gt': 100}

In [108]:
Q(**dct)

Q(**{'description__regex': '^S', 'pamount__gt': 100})

In [109]:
for t in Transaction.objects(Q(**dct)):
    print(t.pdate, t.description, t.pamount)

2024-01-21 Safeway 175.0


Yep!

### Select Method

#### Select Transactions

All transactions:

In [110]:
for t in DB.select(Transaction):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 income:yoyodyne assets:bank:checking
2024-01-02 5000.0 expenses:food/expenses:home/expenses:car/expenses:travel/equity income:yoyodyne
2024-02-02 5000.0 income:yoyodyne assets:bank:checking
2024-02-02 5000.0 expenses:food/expenses:home/expenses:car/expenses:travel/equity income:yoyodyne
2024-01-05 500.0 assets:bank:checking expenses:car:payment
2024-02-05 500.0 assets:bank:checking expenses:car:payment
2024-01-10 50.0 liabilities:chase:visa expenses:car:fuel
2024-02-26 60.0 liabilities:chase:visa expenses:car:fuel
2024-01-04 1800.0 assets:bank:checking expenses:home:mortgage
2024-02-04 1800.0 assets:bank:checking expenses:home:mortgage
2024-01-07 75.0 assets:bank:checking expenses:food:groceries
2024-01-21 175.0 liabilities:chase:visa expenses:food:groceries
2024-02-07 75.0 assets:bank:checking expenses:food:groceries
2024-02-21 75.0 liabilities:chase:visa expenses:food:groceries
2024-03-10 50.0 liabilities:chase:visa expenses:home:household
2024-03-12 25.0 expenses:h

By date:

In [111]:
for t in DB.select(Transaction, date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-21 175.0 liabilities:chase:visa expenses:food:groceries


In [112]:
for t in DB.select(Transaction, start_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-02-02 5000.0 income:yoyodyne assets:bank:checking
2024-02-02 5000.0 expenses:food/expenses:home/expenses:car/expenses:travel/equity income:yoyodyne
2024-02-05 500.0 assets:bank:checking expenses:car:payment
2024-02-26 60.0 liabilities:chase:visa expenses:car:fuel
2024-02-04 1800.0 assets:bank:checking expenses:home:mortgage
2024-01-21 175.0 liabilities:chase:visa expenses:food:groceries
2024-02-07 75.0 assets:bank:checking expenses:food:groceries
2024-02-21 75.0 liabilities:chase:visa expenses:food:groceries
2024-03-10 50.0 liabilities:chase:visa expenses:home:household
2024-03-12 25.0 expenses:home:household liabilities:chase:visa


In [113]:
for t in DB.select(Transaction, end_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 income:yoyodyne assets:bank:checking
2024-01-02 5000.0 expenses:food/expenses:home/expenses:car/expenses:travel/equity income:yoyodyne
2024-01-05 500.0 assets:bank:checking expenses:car:payment
2024-01-10 50.0 liabilities:chase:visa expenses:car:fuel
2024-01-04 1800.0 assets:bank:checking expenses:home:mortgage
2024-01-07 75.0 assets:bank:checking expenses:food:groceries
2024-01-21 175.0 liabilities:chase:visa expenses:food:groceries


By amount:

In [114]:
for t in DB.select(Transaction, amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-07 75.0 assets:bank:checking expenses:food:groceries
2024-02-07 75.0 assets:bank:checking expenses:food:groceries
2024-02-21 75.0 liabilities:chase:visa expenses:food:groceries


In [115]:
lst = DB.select(Transaction, amount=75)

In [116]:
all(t.pamount == 75 for t in lst)

True

In [117]:
for t in DB.select(Transaction, max_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-10 50.0 liabilities:chase:visa expenses:car:fuel
2024-02-26 60.0 liabilities:chase:visa expenses:car:fuel
2024-01-07 75.0 assets:bank:checking expenses:food:groceries
2024-02-07 75.0 assets:bank:checking expenses:food:groceries
2024-02-21 75.0 liabilities:chase:visa expenses:food:groceries
2024-03-10 50.0 liabilities:chase:visa expenses:home:household
2024-03-12 25.0 expenses:home:household liabilities:chase:visa


In [118]:
for t in DB.select(Transaction, min_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 income:yoyodyne assets:bank:checking
2024-01-02 5000.0 expenses:food/expenses:home/expenses:car/expenses:travel/equity income:yoyodyne
2024-02-02 5000.0 income:yoyodyne assets:bank:checking
2024-02-02 5000.0 expenses:food/expenses:home/expenses:car/expenses:travel/equity income:yoyodyne
2024-01-05 500.0 assets:bank:checking expenses:car:payment
2024-02-05 500.0 assets:bank:checking expenses:car:payment
2024-01-04 1800.0 assets:bank:checking expenses:home:mortgage
2024-02-04 1800.0 assets:bank:checking expenses:home:mortgage
2024-01-07 75.0 assets:bank:checking expenses:food:groceries
2024-01-21 175.0 liabilities:chase:visa expenses:food:groceries
2024-02-07 75.0 assets:bank:checking expenses:food:groceries
2024-02-21 75.0 liabilities:chase:visa expenses:food:groceries


By description:

In [119]:
for t in DB.select(Transaction, description = r'^s'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-01-10 50.0 Shell Oil liabilities:chase:visa expenses:car:fuel
2024-02-26 60.0 Shell Oil liabilities:chase:visa expenses:car:fuel
2024-01-07 75.0 Safeway assets:bank:checking expenses:food:groceries
2024-01-21 175.0 Safeway liabilities:chase:visa expenses:food:groceries
2024-02-07 75.0 Safeway assets:bank:checking expenses:food:groceries
2024-02-21 75.0 Safeway liabilities:chase:visa expenses:food:groceries


In [120]:
for t in DB.select(Transaction, comment=r'budget'):
    print(t.pdate, t.pamount, t.description, t.comment, t.pcredit, t.pdebit)

2024-01-02 5000.0 fill envelopes budget: expenses:food/expenses:home/expenses:car/expenses:travel/equity income:yoyodyne
2024-02-02 5000.0 fill envelopes budget: expenses:food/expenses:home/expenses:car/expenses:travel/equity income:yoyodyne


By account:

In [121]:
for t in DB.select(Transaction, debit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-04 1800.0 assets:bank:checking expenses:home:mortgage
2024-02-04 1800.0 assets:bank:checking expenses:home:mortgage


In [122]:
for t in DB.select(Transaction, credit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

Some random combinations

In [123]:
for t in DB.select(Transaction, description = r'^s', min_amount=100):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-01-21 175.0 Safeway liabilities:chase:visa expenses:food:groceries


In [124]:
for t in DB.select(Transaction, start_date = date(2024,2,1), credit='visa'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-02-26 60.0 Shell Oil liabilities:chase:visa expenses:car:fuel
2024-02-21 75.0 Safeway liabilities:chase:visa expenses:food:groceries
2024-03-10 50.0 Home Depot liabilities:chase:visa expenses:home:household


#### Select Entries

All entries:

In [125]:
len(DB.select(Entry))

40

In [126]:
for e in DB.select(Entry):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 assets:bank:checking 5000.0 Column.dr
2024-01-02 income:yoyodyne 5000.0 Column.cr
2024-01-02 income:yoyodyne 5000.0 Column.dr
2024-01-02 expenses:food 500.0 Column.cr
2024-01-02 expenses:home 2500.0 Column.cr
2024-01-02 expenses:car 1000.0 Column.cr
2024-01-02 expenses:travel 500.0 Column.cr
2024-01-02 equity 500.0 Column.cr
2024-02-02 assets:bank:checking 5000.0 Column.dr
2024-02-02 income:yoyodyne 5000.0 Column.cr
2024-02-02 income:yoyodyne 5000.0 Column.dr
2024-02-02 expenses:food 500.0 Column.cr
2024-02-02 expenses:home 2500.0 Column.cr
2024-02-02 expenses:car 1000.0 Column.cr
2024-02-02 expenses:travel 500.0 Column.cr
2024-02-02 equity 500.0 Column.cr
2024-01-05 expenses:car:payment 500.0 Column.dr
2024-01-05 assets:bank:checking 500.0 Column.cr
2024-02-05 expenses:car:payment 500.0 Column.dr
2024-02-05 assets:bank:checking 500.0 Column.cr
2024-01-10 expenses:car:fuel 50.0 Column.dr
2024-01-10 liabilities:chase:visa 50.0 Column.cr
2024-02-26 expenses:car:fuel 60.0 Colum

By date:

In [127]:
for e in DB.select(Entry, date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-01-05 expenses:car:payment 500.0 Column.dr
2024-01-05 assets:bank:checking 500.0 Column.cr


In [128]:
for e in DB.select(Entry, start_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-02-02 assets:bank:checking 5000.0 Column.dr
2024-02-02 income:yoyodyne 5000.0 Column.cr
2024-02-02 income:yoyodyne 5000.0 Column.dr
2024-02-02 expenses:food 500.0 Column.cr
2024-02-02 expenses:home 2500.0 Column.cr
2024-02-02 expenses:car 1000.0 Column.cr
2024-02-02 expenses:travel 500.0 Column.cr
2024-02-02 equity 500.0 Column.cr
2024-01-05 expenses:car:payment 500.0 Column.dr
2024-01-05 assets:bank:checking 500.0 Column.cr
2024-02-05 expenses:car:payment 500.0 Column.dr
2024-02-05 assets:bank:checking 500.0 Column.cr
2024-01-10 expenses:car:fuel 50.0 Column.dr
2024-01-10 liabilities:chase:visa 50.0 Column.cr
2024-02-26 expenses:car:fuel 60.0 Column.dr
2024-02-26 liabilities:chase:visa 60.0 Column.cr
2024-02-04 expenses:home:mortgage 1800.0 Column.dr
2024-02-04 assets:bank:checking 1800.0 Column.cr
2024-01-07 expenses:food:groceries 75.0 Column.dr
2024-01-07 assets:bank:checking 75.0 Column.cr
2024-01-21 expenses:food:groceries 175.0 Column.dr
2024-01-21 liabilities:chase:visa 17

In [129]:
for e in DB.select(Entry, end_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 assets:bank:checking 5000.0 Column.dr
2024-01-02 income:yoyodyne 5000.0 Column.cr
2024-01-02 income:yoyodyne 5000.0 Column.dr
2024-01-02 expenses:food 500.0 Column.cr
2024-01-02 expenses:home 2500.0 Column.cr
2024-01-02 expenses:car 1000.0 Column.cr
2024-01-02 expenses:travel 500.0 Column.cr
2024-01-02 equity 500.0 Column.cr
2024-01-05 expenses:car:payment 500.0 Column.dr
2024-01-05 assets:bank:checking 500.0 Column.cr
2024-01-04 expenses:home:mortgage 1800.0 Column.dr
2024-01-04 assets:bank:checking 1800.0 Column.cr


By amount:

In [130]:
for e in DB.select(Entry, amount=900):
    print(e.date, e.account, e.amount, e.column)

In [131]:
for e in DB.select(Entry, max_amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 expenses:food 500.0 Column.cr
2024-01-02 expenses:travel 500.0 Column.cr
2024-01-02 equity 500.0 Column.cr
2024-02-02 expenses:food 500.0 Column.cr
2024-02-02 expenses:travel 500.0 Column.cr
2024-02-02 equity 500.0 Column.cr
2024-01-05 expenses:car:payment 500.0 Column.dr
2024-01-05 assets:bank:checking 500.0 Column.cr
2024-02-05 expenses:car:payment 500.0 Column.dr
2024-02-05 assets:bank:checking 500.0 Column.cr
2024-01-10 expenses:car:fuel 50.0 Column.dr
2024-01-10 liabilities:chase:visa 50.0 Column.cr
2024-02-26 expenses:car:fuel 60.0 Column.dr
2024-02-26 liabilities:chase:visa 60.0 Column.cr
2024-01-07 expenses:food:groceries 75.0 Column.dr
2024-01-07 assets:bank:checking 75.0 Column.cr
2024-01-21 expenses:food:groceries 175.0 Column.dr
2024-01-21 liabilities:chase:visa 175.0 Column.cr
2024-02-07 expenses:food:groceries 75.0 Column.dr
2024-02-07 assets:bank:checking 75.0 Column.cr
2024-02-21 expenses:food:groceries 75.0 Column.dr
2024-02-21 liabilities:chase:visa 75.0 Co

In [132]:
for e in DB.select(Entry, min_amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 assets:bank:checking 5000.0 Column.dr
2024-01-02 income:yoyodyne 5000.0 Column.cr
2024-01-02 income:yoyodyne 5000.0 Column.dr
2024-01-02 expenses:home 2500.0 Column.cr
2024-01-02 expenses:car 1000.0 Column.cr
2024-02-02 assets:bank:checking 5000.0 Column.dr
2024-02-02 income:yoyodyne 5000.0 Column.cr
2024-02-02 income:yoyodyne 5000.0 Column.dr
2024-02-02 expenses:home 2500.0 Column.cr
2024-02-02 expenses:car 1000.0 Column.cr
2024-01-04 expenses:home:mortgage 1800.0 Column.dr
2024-01-04 assets:bank:checking 1800.0 Column.cr
2024-02-04 expenses:home:mortgage 1800.0 Column.dr
2024-02-04 assets:bank:checking 1800.0 Column.cr


By account:

In [133]:
for e in DB.select(Entry, account='medical'):
    print(e.date, e.account, e.amount, e.column)

By column:

In [134]:
for e in DB.select(Entry, column='credit'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 income:yoyodyne 5000.0 Column.cr
2024-01-02 expenses:food 500.0 Column.cr
2024-01-02 expenses:home 2500.0 Column.cr
2024-01-02 expenses:car 1000.0 Column.cr
2024-01-02 expenses:travel 500.0 Column.cr
2024-01-02 equity 500.0 Column.cr
2024-02-02 income:yoyodyne 5000.0 Column.cr
2024-02-02 expenses:food 500.0 Column.cr
2024-02-02 expenses:home 2500.0 Column.cr
2024-02-02 expenses:car 1000.0 Column.cr
2024-02-02 expenses:travel 500.0 Column.cr
2024-02-02 equity 500.0 Column.cr
2024-01-05 assets:bank:checking 500.0 Column.cr
2024-02-05 assets:bank:checking 500.0 Column.cr
2024-01-10 liabilities:chase:visa 50.0 Column.cr
2024-02-26 liabilities:chase:visa 60.0 Column.cr
2024-01-04 assets:bank:checking 1800.0 Column.cr
2024-02-04 assets:bank:checking 1800.0 Column.cr
2024-01-07 assets:bank:checking 75.0 Column.cr
2024-01-21 liabilities:chase:visa 175.0 Column.cr
2024-02-07 assets:bank:checking 75.0 Column.cr
2024-02-21 liabilities:chase:visa 75.0 Column.cr
2024-03-10 liabilities:ch

In [135]:
for e in DB.select(Entry, column='debit'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 assets:bank:checking 5000.0 Column.dr
2024-01-02 income:yoyodyne 5000.0 Column.dr
2024-02-02 assets:bank:checking 5000.0 Column.dr
2024-02-02 income:yoyodyne 5000.0 Column.dr
2024-01-05 expenses:car:payment 500.0 Column.dr
2024-02-05 expenses:car:payment 500.0 Column.dr
2024-01-10 expenses:car:fuel 50.0 Column.dr
2024-02-26 expenses:car:fuel 60.0 Column.dr
2024-01-04 expenses:home:mortgage 1800.0 Column.dr
2024-02-04 expenses:home:mortgage 1800.0 Column.dr
2024-01-07 expenses:food:groceries 75.0 Column.dr
2024-01-21 expenses:food:groceries 175.0 Column.dr
2024-02-07 expenses:food:groceries 75.0 Column.dr
2024-02-21 expenses:food:groceries 75.0 Column.dr
2024-03-10 expenses:home:household 50.0 Column.dr
2024-03-12 liabilities:chase:visa 25.0 Column.dr


By tag:

In [136]:
for e in DB.select(Entry, tag='unpaired'):
    print(e.date, e.account, e.amount, e.column)

### Serializing Objects

In [137]:
import json
from bson.objectid import ObjectId
import datetime

In [138]:
lst = DB.select(Transaction, start_date = date(2024,2,1), credit='visa')

In [139]:
lst[0].to_json()

'{"_id": {"$oid": "688d869094e4e84ee4a8b9a8"}, "description": "Shell Oil", "comment": "", "tags": [], "entries": [{"$oid": "688d869094e4e84ee4a8b9a6"}, {"$oid": "688d869094e4e84ee4a8b9a7"}], "pdate": {"$date": 1708905600000}, "pdebit": "expenses:car:fuel", "pcredit": "liabilities:chase:visa", "pamount": 60.0}'

In [140]:
type(lst[0])

dexter.DB.Transaction

In [141]:
obj = Transaction.objects.as_pymongo()[0]

In [142]:
type(obj)

dict

In [143]:
obj

{'_id': ObjectId('688d869094e4e84ee4a8b98b'),
 'description': 'Jan paycheck',
 'comment': '',
 'tags': [],
 'entries': [ObjectId('688d869094e4e84ee4a8b989'),
  ObjectId('688d869094e4e84ee4a8b98a')],
 'pdate': datetime.datetime(2024, 1, 2, 0, 0),
 'pdebit': 'assets:bank:checking',
 'pcredit': 'income:yoyodyne',
 'pamount': 5000.0}

In [144]:
s = lst[0].to_json()

In [145]:
json.loads(s)

{'_id': {'$oid': '688d869094e4e84ee4a8b9a8'},
 'description': 'Shell Oil',
 'comment': '',
 'tags': [],
 'entries': [{'$oid': '688d869094e4e84ee4a8b9a6'},
  {'$oid': '688d869094e4e84ee4a8b9a7'}],
 'pdate': {'$date': 1708905600000},
 'pdebit': 'expenses:car:fuel',
 'pcredit': 'liabilities:chase:visa',
 'pamount': 60.0}

In [146]:
Transaction.from_json(s)

<Transaction: <Tr 2024-02-26 liabilities:chase:visa -> expenses:car:fuel $60.0 Shell Oil []>>

In [147]:
s = 'account: {...:...}'

In [148]:
s.find(':')

7

In [149]:
s[:s.find(':')]

'account'

In [150]:
s[s.find(':'):]

': {...:...}'

### Indexes

We want a field in Entry documents that serves as a unique ID so we can tell if an item was imported before.

MongoEngine has a UUID field.
* how is it computed?  is it a hash of all the other field values?
* when is it computed?  when the object is made, or when it is saved?

In [151]:
from mongoengine import *

In [152]:
class Foo(Document):
    name = StringField()
    amount = FloatField()
    uid = UUIDField(binary=False)

In [153]:
f = Foo(name='Fred', amount=10)

Just declaring it is not enough to give it a value:

In [154]:
f.uid is None

True

In [155]:
f.save()

<Foo: Foo object>

This model has an index.  The `#` means it's a "hashed index" but no discussion of what that means or why we'd want one (over say a text index that we compute ourselves).

In [156]:
class Bar(Document):
    name = StringField()
    amount = FloatField()
    uid = UUIDField(binary=False)
    meta = {
        'indexes': ['#uid']
    }

In [157]:
b1 = Bar(name='george', amount=20, uid='123')

In [158]:
b1.uid

'123'

Ah -- the UUID is created when the document is saved (the same was true for Foo, above, if a `uid` value passed to the constructor).

```
b1.save()
...
ValidationError: ValidationError (Bar:None) (Could not convert to UUID: badly formed hexadecimal UUID string: ['uid'])
```

So how do we make a UUID?  Do we care?  Why not just use our hashed strings?

#### UUID

It's in the PyMongo docs (and we had to specify how they're represented when we made the DB connection).  It's also a Python library.

In [159]:
from uuid import uuid4

In [160]:
b2 = Bar(name='ringo', amount=30, uid=uuid4())

In [161]:
b2.uid

UUID('e20911c3-6f2d-4ebc-bf34-156ba09deccb')

In [162]:
b2.save()

<Bar: Bar object>

In [163]:
x = b2.uid

In [164]:
b3 = Bar(name='paul', amount=40, uid=x)

In [165]:
b3.save()

<Bar: Bar object>

In [166]:
b3.uid == b2.uid

True

In [167]:
uuid4()

UUID('09e223f1-e1fc-48ed-b5f1-2431c5cefa89')

So just defining an index isn't enough to make it unique.

### Unique

In [168]:
class FooBar(Document):
    name = StringField()
    amount = FloatField()
    uid = StringField(unique=True)

In [169]:
f1 = FooBar(name='Fred', amount=100, uid='bedrock')

In [170]:
f2 = FooBar(name='Barney', amount=200, uid='bedrock')

If uncommented this cell will add a record to the database.  To reset the database to its original condition run pytest again.

In [171]:
# f1.save()

In [172]:
# try:
#     f2.save()
# except NotUniqueError as err:
#     print(err)

In [173]:
FooBar._meta

{'abstract': False,
 'max_documents': None,
 'max_size': None,
 'ordering': [],
 'indexes': [],
 'id_field': 'id',
 'index_background': False,
 'index_opts': None,
 'delete_rules': None,
 'allow_inheritance': None,
 'collection': 'foo_bar',
 'index_specs': [{'fields': [('uid', 1)], 'unique': True, 'sparse': False}]}

Awesome!  Just defining a field as unique is enough to have MongoEngine create an index.  Don't know (and don't care, yet, at least) about the ramifications of `sparse = False`.

> checked PyMongo docs, it's not what we think, and not something we want (even though we can have it by specifying `sparse=True` in the column spec)

### Aggregation

In [174]:
Entry.objects.sum('amount')

50370.0

In [175]:
Entry.objects(column='credit')

[<Entry: <En 2024-01-02 income:yoyodyne -$5000.0 []>>, <Entry: <En 2024-01-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-01-02 expenses:home -$2500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-01-02 expenses:car -$1000.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-01-02 expenses:travel -$500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-01-02 equity -$500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-02-02 income:yoyodyne -$5000.0 []>>, <Entry: <En 2024-02-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-02-02 expenses:home -$2500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-02-02 expenses:car -$1000.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-02-02 expenses:travel -$500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-02-02 equity -$500.0 [<Tag.B: '#budget'>]>>, <Entry: <En 2024-01-05 assets:bank:checking -$500.0 []>>, <Entry: <En 2024-02-05 assets:bank:checking -$500.0 []>>, <Entry: <En 2024-01-10 liabilities:chase:visa -$50.0 []>>, <Entry: <En 2024-02-26 liabilities

In [176]:
Entry.objects(column='credit').sum('amount')

25185.0

In [177]:
Entry.objects(Q(account="groceries") & Q(column='credit')).sum('amount')

0

In [178]:
for e in Entry.objects(column='credit'):
    print(e)

<En 2024-01-02 income:yoyodyne -$5000.0 []>
<En 2024-01-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>
<En 2024-01-02 expenses:home -$2500.0 [<Tag.B: '#budget'>]>
<En 2024-01-02 expenses:car -$1000.0 [<Tag.B: '#budget'>]>
<En 2024-01-02 expenses:travel -$500.0 [<Tag.B: '#budget'>]>
<En 2024-01-02 equity -$500.0 [<Tag.B: '#budget'>]>
<En 2024-02-02 income:yoyodyne -$5000.0 []>
<En 2024-02-02 expenses:food -$500.0 [<Tag.B: '#budget'>]>
<En 2024-02-02 expenses:home -$2500.0 [<Tag.B: '#budget'>]>
<En 2024-02-02 expenses:car -$1000.0 [<Tag.B: '#budget'>]>
<En 2024-02-02 expenses:travel -$500.0 [<Tag.B: '#budget'>]>
<En 2024-02-02 equity -$500.0 [<Tag.B: '#budget'>]>
<En 2024-01-05 assets:bank:checking -$500.0 []>
<En 2024-02-05 assets:bank:checking -$500.0 []>
<En 2024-01-10 liabilities:chase:visa -$50.0 []>
<En 2024-02-26 liabilities:chase:visa -$60.0 []>
<En 2024-01-04 assets:bank:checking -$1800.0 []>
<En 2024-02-04 assets:bank:checking -$1800.0 []>
<En 2024-01-07 assets:bank:checking -$

In [179]:
{'a': 0} | {'b': 1}

{'a': 0, 'b': 1}